In [144]:
#MODEL = "gpt-4o-2024-08-06"
#MODEL = "gpt-4o-mini"
MODEL = "gpt-3.5-turbo"
#prompt = [{"role": "user", "content": ""}]
loops = 100
file = f"{MODEL}/4"

import os
from openai import OpenAI
import pandas as pd
from IPython.display import display_markdown
import copy

client = OpenAI()
df_path = os.path.join(os.getcwd(),"completions")
chat = False


def transform_roles(chat,assistant_transform_type = "user-assistant"):
    prompt_messages = copy.deepcopy(chat)
    if assistant_transform_type == "user-assistant":
      i = 1
      while i <= len(prompt_messages) and prompt_messages[-i]["role"] != "system":
        if i%2 == 1:
          prompt_messages[-i]["role"] = "user"
        else:
          prompt_messages[-i]["role"] = "assistant"
        i += 1
    else:
      for prompt_message in prompt_messages:
        if prompt_message["role"] == "assistant":
          prompt_message["role"] = assistant_transform_type
    return prompt_messages

In [145]:
# Load completions from json, if continuing with a file generated in the past

try:
    with open(os.path.join(df_path,file), "r") as f:
        completions = pd.read_json(f, lines=True)

# If not, create a new completions database

except FileNotFoundError:
    completions = pd.DataFrame([],columns=['Response', 'Eval', 'Data', 'Instruction'])
    print(completions)

In [142]:
for i in range(loops):
    completion = client.chat.completions.create(
        messages = prompt,
        model = MODEL,
        )
    response = completion.choices[0].message.content

    data = {'Response': response, 'Eval': "", 'Data': {}, 'Instruction': prompt}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict()}
    completions = pd.concat([completions, data],ignore_index=True)
    completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [146]:
completions

,Response,Eval,Data,Instruction
0,Of course! Here are some of the latest technol...,,{'Completion': {'id': 'chatcmpl-9wckBhOlxx2fTl...,"[{'role': 'user', 'content': 'Hello! Can you p..."
1,Of course! Here are some of the latest technol...,,{'Completion': {'id': 'chatcmpl-9wckEolODVZ6Bw...,"[{'role': 'user', 'content': 'Hello! Can you p..."
2,Of course! Here are some of the latest technol...,,{'Completion': {'id': 'chatcmpl-9wckIEBQmnI5bM...,"[{'role': 'user', 'content': 'Hello! Can you p..."
3,Of course! Here are some of the latest technol...,,{'Completion': {'id': 'chatcmpl-9wckMdjpkyM6hr...,"[{'role': 'user', 'content': 'Hello! Can you p..."
4,Sure! Here are some of the latest technology t...,,{'Completion': {'id': 'chatcmpl-9wckRpXBXjetNh...,"[{'role': 'user', 'content': 'Hello! Can you p..."
...,...,...,...,...
95,Of course! Here are some of the latest technol...,,{'Completion': {'id': 'chatcmpl-9wcqYPbN0e1N8d...,"[{'role': 'user', 'content': 'Hello! Can you p..."
96,Of course! Here are some of the latest technol...,,{'Completion': {'id': 'chatcmpl-9wcqbljVET8eS1...,"[{'role': 'user', 'content': 'Hello! Can you p..."
97,Of course! Here are some of the latest technol...,,{'Completion': {'id': 'chatcmpl-9wcqfLVCDDfwCR...,"[{'role': 'user', 'content': 'Hello! Can you p..."
98,Of course! Here are some of the latest technol...,,{'Completion': {'id': 'chatcmpl-9wcqjdLzKuK8Xk...,"[{'role': 'user', 'content': 'Hello! Can you p..."


In [149]:
n = 60
completions.iloc[n]

Response       Absolutely! Here are some of the latest techno...
Eval                                        First alphabetically
Data           {'Completion': {'id': 'chatcmpl-9wco1dtPpD93v4...
Instruction    [{'role': 'user', 'content': 'Hello! Can you p...
Name: 60, dtype: object

In [148]:
completions.loc[n,"Eval"] = "First alphabetically"
completions.to_json(os.path.join(df_path, file), lines=True, orient="records")

In [150]:
tag = "First alphabetically"
selected_response = completions[completions["Eval"].apply(lambda x: x.startswith(tag))].iloc[0]["Response"]
if chat:
    messages = copy.deepcopy(completions[completions["Eval"].apply(lambda x: x.startswith(tag))].iloc[0]["Instruction"])
    messages.append({"role": "assistant", "content": selected_response_response})
    prompt = transform_roles(messages)
else:
    prompt = [{"role": "user", "content": selected_response}]
print(prompt)

[{'role': 'user', 'content': 'Absolutely! Here are some of the latest technology trends that are shaping the industry:\n\n1. Artificial Intelligence (AI) and Machine Learning: AI and machine learning are leading the way in transforming industries such as healthcare, finance, and automation. They are being used to optimize processes, improve decision-making, and create personalized experiences for users.\n\n2. Internet of Things (IoT): IoT has become increasingly prevalent, with more devices being connected to the internet for enhanced communication and data collection. This has resulted in smart homes, smart cities, and connected vehicles, among other applications.\n\n3. 5G Technology: The rollout of 5G technology is revolutionizing connectivity with faster speeds, lower latency, and increased capacity. This is enabling advancements in areas such as augmented reality, virtual reality, and autonomous vehicles.\n\n4. Edge Computing: Edge computing brings processing power closer to the so